In [0]:
import pandas as pd

In [105]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_csv('/content/drive/My Drive/Data Science/Churn.csv' ,index_col="RowNumber")

In [119]:
df.corr()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
CustomerId,1.000000,0.005308,0.009497,-0.014883,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
CreditScore,0.005308,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,0.009497,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.014883,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,-0.012419,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.016972,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,-0.014025,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.001665,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,0.015271,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097
Exited,-0.006248,-0.027094,0.285323,-0.014001,0.118533,-0.047820,-0.007138,-0.156128,0.012097,1.000000


In [107]:
df.head(5)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
df1 = df.drop(['CustomerId','Surname'],axis=1)

In [109]:
df1.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,
1,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
#one hot encode
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
df1['Geography']= label_encoder.fit_transform(df1['Geography'])
df1['Gender']= label_encoder.fit_transform(df1['Gender'])

In [0]:
df1 = pd.get_dummies(df1, columns=['Geography'])

In [164]:
df1.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_0', 'Geography_1', 'Geography_2'],
      dtype='object')

In [0]:
#Distinguish the feature and target set
Y = df1['Exited']  #Target
X = df1.drop('Exited',1) #Feature

In [0]:
# Splitting the data into train and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 101)

In [0]:
#Normalize
from sklearn.preprocessing import Normalizer
Normalize = Normalizer()
X_train =  Normalize.fit_transform(X_train) 
X_test =  Normalize.fit_transform(X_test)

In [0]:
#initialize and build the model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

In [180]:
model = Sequential()
model.add(Dense(200,input_dim=12, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(50, activation='softmax'))
model.add(Dense(1, activation='relu'))
opt = SGD(lr=0.001, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=12, activation="relu", kernel_initializer="he_uniform", units=6)`
  


In [181]:
model.fit(X_train,Y_train,epochs=5,validation_data=(X_test, Y_test))

Train on 7000 samples, validate on 3000 samples
Epoch 1/5
7000/7000 [==============================] - 4s 537us/step - loss: 0.5046 - acc: 0.7979 - val_loss: 0.5060 - val_acc: 0.7927
Epoch 2/5
7000/7000 [==============================] - 1s 81us/step - loss: 0.4971 - acc: 0.7979 - val_loss: 0.5046 - val_acc: 0.7927
Epoch 3/5
7000/7000 [==============================] - 1s 88us/step - loss: 0.4957 - acc: 0.7979 - val_loss: 0.5049 - val_acc: 0.7927
Epoch 4/5
7000/7000 [==============================] - 1s 100us/step - loss: 0.4962 - acc: 0.7979 - val_loss: 0.5046 - val_acc: 0.7927
Epoch 5/5
7000/7000 [==============================] - 1s 88us/step - loss: 0.4955 - acc: 0.7979 - val_loss: 0.5045 - val_acc: 0.7927


In [116]:
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [117]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,y_pred)
cm

array([[2388,    0],
       [ 612,    0]])